# **Dependnecies and setting up**
   - Make sure you read comments to know which ones to run

In [12]:
## run only in colab after cloning repo to be able to access other files easily
import sys
import os
GIT_PATH = '/content/svhn_dl'

if GIT_PATH not in sys.path:
  sys.path.append(GIT_PATH)


In [7]:
# DO NOT RUN THIS CELL IF RUNNING LOCALLY
# run to install dependencide IN COLAB if running locally use next cell
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install numpy matplotlib scipy scikit-learn jupyter

Looking in indexes: https://download.pytorch.org/whl/cu121


In [8]:
#!pip install -r ./requirements.txt

In [11]:
#run to download dataset unless already downloaded locally
# must have already cloned repo to colab workspace
!cd svhn_dl
!mkdir -p data
!cd data && wget http://ufldl.stanford.edu/housenumbers/train_32x32.mat
!cd data && wget http://ufldl.stanford.edu/housenumbers/test_32x32.mat

--2025-12-11 04:02:29--  http://ufldl.stanford.edu/housenumbers/train_32x32.mat
Resolving ufldl.stanford.edu (ufldl.stanford.edu)... 171.64.68.10
Connecting to ufldl.stanford.edu (ufldl.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 182040794 (174M) [text/plain]
Saving to: ‘train_32x32.mat.1’

train_32x32.mat.1   100%[===================>] 173.61M  5.02MB/s    in 29s     

2025-12-11 04:02:58 (6.03 MB/s) - ‘train_32x32.mat.1’ saved [182040794/182040794]

--2025-12-11 04:02:58--  http://ufldl.stanford.edu/housenumbers/test_32x32.mat
Resolving ufldl.stanford.edu (ufldl.stanford.edu)... 171.64.68.10
Connecting to ufldl.stanford.edu (ufldl.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64275384 (61M) [text/plain]
Saving to: ‘test_32x32.mat.1’

test_32x32.mat.1    100%[===================>]  61.30M  8.10MB/s    in 8.2s    

2025-12-11 04:03:06 (7.43 MB/s) - ‘test_32x32.mat.1’ saved [642

In [13]:
# set path to data directory
DATA_PATH = './data'

In [15]:
# imports

import torch
import torch.nn as nn
#import sys
#import os

In [ ]:
# setting up initial CNN for testing

class SimpleCNN(nn.Module):

  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3,32, kernel_size=5)
    self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
    self.pool = nn.MaxPool2d(2,2)
    self.fc1 = nn.Linear(64*5*5, 128)
    self.fc2 = nn.Linear(128, 10)
    self.relu = nn.RelU()

  def forward(self, x):
    x = self.pool(self.relu(self.conv1(x)))
    x = self.pool(self.relu(self.conv2(x)))
    x = self.relU(self.fc1(x))
    x = self.fc2(x)
    return x

